In [1]:
import pandas as pd
import numpy as np
import MySQLdb
import os
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

In [2]:
db = MySQLdb.connect(host='104.154.94.28',db='loganp',\
                     read_default_file="~/.my.cnf",\
                     autocommit=True,\
                     local_infile = 1)
c=db.cursor()

In [3]:
def get_targets(ra,dec,ang_diam,project,name,dist=5000,con=db,database='master_gaia_database'):
    """
    Given an RA/Dec array of any size, returns a Pandas databse of
    the targets from the master database that fall within 
    a circle of specified angular diameter of the given RA/Decs
    All angles in degrees
    Args:
        ra,dec (array, float [deg]): arrays of pointing coordinates in decimal degrees
        ang_diam (float [deg]): angular size of the diameter of the beam you are simulating.  Default of 0.8 deg
                          is slightly smaller than the MeerKAT beam in L-band to provide a conservative estimate
        project (string): project name that this pointing came from
        name (string): name of the primary beam target
        dist (float, [pc]): depth of the desired query in parsecs.  Default is set larger than the largest distance
                          in the master database to return essentially no distance cut
        con (MySQL connection object): the connection you would like to use for the retrieval
        database (str): the database to query.  Default is the master database of all Gaia targets in our program
    Returns:
        Pandas dataframe of every object in the database meeting criteria
    """
    index = range(len(ra))
    appended_data = [] #make a list to store dataframes
    for r,d,i in zip(ra,dec,index):
        string = 'SELECT * FROM '+str(database)+' \
                    WHERE POWER((ra-('+str(r)+')),2) + POWER((decl - ('+str(d)+')),2) < '+str((ang_diam/2.)**2)+' \
                    AND `dist.c` <= '+str(dist)+';'
        dataframe = pd.read_sql(string, con=con)
        dataframe['project']=project
        dataframe['name']=name
        # store DataFrame in list
        appended_data.append(dataframe)
        print "I've done ",i+1," of ",len(ra)," total pointings"
    targets = pd.concat(appended_data, axis=0)
    return targets

In [8]:
clusters = ['Liller 1','NGC 6388','Omega Centauri','2MASS-GC01','2MASS-GC02']

In [13]:
from astroquery.simbad import Simbad
from astropy.table import QTable, Table
table=pd.DataFrame()

for c in clusters:
    t=Simbad.query_object(c)
    t = Table(t)
    t1=Table()
    t1['RA'] = t['RA']
    t1['DEC'] = t['DEC']
    t1['NAME'] = t['MAIN_ID']
    t1=t1.to_pandas()
    table=table.append(t1)
    
radeg,decdeg = np.zeros(len(table)),np.zeros(len(table))
for r,d,i in zip(table['RA'],table['DEC'],range(len(table))):
    sc = SkyCoord(ra=r, dec=d, unit=(u.hourangle, u.deg), equinox='J2000.0')
    radeg[i],decdeg[i] = sc.ra.degree,sc.dec.degree
table['ra_deg'],table['dec_deg'] = radeg,decdeg
table

,RA,DEC,NAME,ra_deg,dec_deg
0,17 33 24.50,-33 23 20.0,C 1730-333,263.352083,-33.388889
0,17 36 17.461,-44 44 08.34,NGC 6388,264.072754,-44.735650
0,13 26 47.28,-47 28 46.1,NGC 5139,201.697000,-47.479472
0,18 08 21.8,-19 49 47,NAME 2MASS-GC01,272.090833,-19.829722
0,18 09 36.5,-20 46 44,NAME 2MASS-GC02,272.402083,-20.778889


In [14]:
table.to_csv('Trapum_GC_targets.csv',index=False)

In [16]:
count=0
index = np.arange(0,len(table),1)
for i in index:
    print 'Doing index ',i,' of ',np.max(index)
    targets = get_targets([table['ra_deg'].values[i]],[table['dec_deg'].values[i]],1.5,'TRAPUM_GC',\
                          table['NAME'].values[i])
    if count == 0:
        targets.to_csv('trapum_GC.csv',index=False)
    else:
        targets.to_csv('trapum_GC.csv',mode='a',header=False,index=False)

    count=count+1

Doing index  0  of  4
I've done  1  of  1  total pointings
Doing index  1  of  4
I've done  1  of  1  total pointings
Doing index  2  of  4
I've done  1  of  1  total pointings
Doing index  3  of  4
I've done  1  of  1  total pointings
Doing index  4  of  4
I've done  1  of  1  total pointings
